In [111]:
import pandas as pd
import ast

In [112]:
nrel = pd.read_csv("dSTAC_US.csv", dtype={"state_fips": "object", "county_fips": "object"})
nrel.head()

,state_abbr,state_fips,county_fips,sector_abbr,energy_value_us_dollars_per_kwh,percent_customers_with_nonzero_sys_size,avg_roof_sqft_customers_with_nonzero_sys_size,avg_roof_sqft_total,hourly_capacity_factor,hourly_capacity_factor_scalar
0,AL,1,1,com,0.0911,0.99,2473.0,4196.0,"[0,0,0,0,0,0,0,799,4190,6338,7315,7366,7049,65...",10000
1,AL,1,1,ind,0.1143,1.00,4196.0,4196.0,"[0,0,0,0,0,0,0,799,4190,6338,7315,7366,7049,65...",10000
2,AL,1,1,res,0.1037,0.54,415.0,349.0,"[0,0,0,0,0,0,0,799,4190,6338,7315,7366,7049,65...",10000
3,AL,1,3,com,0.0743,0.56,2596.0,3973.0,"[0,0,0,0,0,0,0,674,1976,2935,3579,1372,4828,55...",10000
4,AL,1,3,ind,0.0715,0.34,6109.0,3973.0,"[0,0,0,0,0,0,0,674,1976,2935,3579,1372,4828,55...",10000


In [113]:
nrel.dtypes

state_abbr                                        object
state_fips                                        object
county_fips                                       object
sector_abbr                                       object
energy_value_us_dollars_per_kwh                   object
percent_customers_with_nonzero_sys_size          float64
avg_roof_sqft_customers_with_nonzero_sys_size    float64
avg_roof_sqft_total                              float64
hourly_capacity_factor                            object
hourly_capacity_factor_scalar                      int64
dtype: object

In [114]:
nrel["state_fips"] = nrel["state_fips"].apply(lambda x: x.zfill(2))
nrel["county_fips"] = nrel["county_fips"].apply(lambda x: x.zfill(3))

In [115]:
full_fips = {
    "Maricopa, AZ": "04013",
    "Los Angeles, CA": "06037",
    "Dade, FL": "12025",
    "Kings, NY": "36047",
    "King, WA": "53033",
    "Cook, IL": "17031",
    "Clark, NV": "32003",
    "Harris, TX": "48201",
    "Dane, WI": "55025",
    "Fairfax, VA": "51059"
}

In [116]:
nrel.dtypes

state_abbr                                        object
state_fips                                        object
county_fips                                       object
sector_abbr                                       object
energy_value_us_dollars_per_kwh                   object
percent_customers_with_nonzero_sys_size          float64
avg_roof_sqft_customers_with_nonzero_sys_size    float64
avg_roof_sqft_total                              float64
hourly_capacity_factor                            object
hourly_capacity_factor_scalar                      int64
dtype: object

In [117]:
nrel["full_fips"] = nrel["state_fips"] + nrel["county_fips"]
nrel["full_fips"]

0       01001
1       01001
2       01001
3       01003
4       01003
        ...  
9309    56043
9310    56043
9311    56045
9312    56045
9313    56045
Name: full_fips, Length: 9314, dtype: object

In [118]:
full_fips.values()

dict_values(['04013', '06037', '12025', '36047', '53033', '17031', '32003', '48201', '55025', '51059'])

In [119]:
nrel = nrel[nrel["full_fips"].isin(full_fips.values())]
nrel.head()

,state_abbr,state_fips,county_fips,sector_abbr,energy_value_us_dollars_per_kwh,percent_customers_with_nonzero_sys_size,avg_roof_sqft_customers_with_nonzero_sys_size,avg_roof_sqft_total,hourly_capacity_factor,hourly_capacity_factor_scalar,full_fips
449,AZ,04,013,com,0.0638,1.00,1880.0,1880.0,"[0,0,0,0,0,0,0,264,2843,5097,6196,7248,7482,73...",10000,04013
450,AZ,04,013,ind,0.0748,1.00,1906.0,1906.0,"[0,0,0,0,0,0,0,264,2843,5097,6196,7248,7482,73...",10000,04013
451,AZ,04,013,res,0.0965,1.00,392.0,392.0,"[0,0,0,0,0,0,0,264,2843,5097,6196,7248,7482,73...",10000,04013
527,CA,06,037,com,0.0957,0.91,6141.0,6107.0,"[0,0,0,0,0,0,0,1157,3799,5654,7018,7541,6990,6...",10000,06037
528,CA,06,037,ind,0.0685,1.00,10828.0,10828.0,"[0,0,0,0,0,0,0,1157,3799,5654,7018,7541,6990,6...",10000,06037


In [120]:
# state_abbr = "AZ"
# df = nrel[nrel["state_abbr"] == state_abbr]
# print(df.head())

# com = ast.literal_eval(df[df["sector_abbr"] == "com"]["hourly_capacity_factor"].values[0])
# ind = ast.literal_eval(df[df["sector_abbr"] == "ind"]["hourly_capacity_factor"].values[0])
# res = ast.literal_eval(df[df["sector_abbr"] == "res"]["hourly_capacity_factor"].values[0])
# print(len(res))

# avg_hourly_capacity_factor = [(com[i] + ind[i] + res[i]) / 3 for i in range(8760)]   # hours per year
# print(avg_hourly_capacity_factor)

def agg_func(x):
    # print(type(x))
    # print(x.values[0])
    # print(type(x.values[0]))
    # print(len(x.values))
    # print(ast.literal_eval(x.values[0]))
    # print(type(ast.literal_eval(x.values[0])))
    # For each list in the group (Series), convert from a string to a list
    lists = [ast.literal_eval(li) for li in x.values]
    # print(lists)
    # print(len(lists))

    # Transpose the lists so that matching indices of the lists are grouped
    # [[1, 2, 3], [4, 5, 6], [7, 8, 9]] -> [[1, 4, 7], [2, 5, 8], [3, 6, 9]]
    transposed = zip(*lists)
    # for x in transposed:
    #     print(x)
    #     print(type(x))
    # evaluated = [ast.literal_eval(li) for li in transposed]
    # print(lists)
    # com = ast.literal_eval(x[x["sector_abbr"] == "com"]["hourly_capacity_factor"].values[0])
    # print(com)
    # print(x)

    # print(type(x))
    # Calculate the average of each group of indices
    return [sum(values) / len(values) for values in transposed]

grouped = nrel.groupby("state_abbr")["hourly_capacity_factor"].agg(agg_func)
print(grouped)
print(type(grouped))

# for state_abbr in df["state_abbr"].unique:


# df.groupby("hourly_capacity_factor").mean()
# nrel["avg_hourly_capacity_factor"] = nrel.apply(lambda x: )
# for state_abbr in nrel["state_abbr"].unique():
#     print(nrel["state_abbr"] == state_abbr)
    # com = nrel[nrel["state_abbr"] == state_abbr]
    # print(com)
    # com = nrel[(nrel["state_abbr"] == state_abbr) and (nrel["sector_abbr"] == "com")]
    # print(com)

state_abbr
AZ    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 264.0, 284...
CA    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1157.0, 37...
IL    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 516.0, 268...
NV    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 563.0, 784...
NY    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 155.0...
TX    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 195.0, 161...
VA    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 142.0...
WA    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1674....
WI    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 120.0, 168...
Name: hourly_capacity_factor, dtype: object
<class 'pandas.core.series.Series'>


In [121]:
grouped.reset_index().index

RangeIndex(start=0, stop=9, step=1)

In [122]:
nrel.index

Index([ 449,  450,  451,  527,  528,  529, 2027, 2028, 2029, 5792, 5793, 5794,
       5909, 5910, 5911, 7758, 7759, 7760, 8390, 8391, 8392, 8795, 8796, 8797,
       8900, 8901, 8902],
      dtype='int64')

In [123]:
nrel.dtypes

state_abbr                                        object
state_fips                                        object
county_fips                                       object
sector_abbr                                       object
energy_value_us_dollars_per_kwh                   object
percent_customers_with_nonzero_sys_size          float64
avg_roof_sqft_customers_with_nonzero_sys_size    float64
avg_roof_sqft_total                              float64
hourly_capacity_factor                            object
hourly_capacity_factor_scalar                      int64
full_fips                                         object
dtype: object

In [124]:
grouped_df = pd.DataFrame(grouped).reset_index(drop=False)
grouped_df.dtypes

state_abbr                object
hourly_capacity_factor    object
dtype: object

In [125]:
# nrel = nrel.join(grouped_df, on="state_abbr")
# full = pd.concat([grouped_df, nrel], join="inner")
# full.head()
grouped.join(nrel, on="state_abbr")

AttributeError: 'Series' object has no attribute 'join'

In [ ]:
# Divide hourly_capacity_factor by hourly_capacity_value_scalar to render generation in the correct unit (kWh/kW)
nrel["real_generation"] = pd.DataFrame(grouped)["hourly_capacity_factor"].apply(lambda x: ast.literal_eval(x))

def func(row):
    # print(row)
    return [hour / row["hourly_capacity_factor_scalar"] for hour in ast.literal_eval(row["hourly_capacity_factor"])]

nrel["real_generation"] = nrel.apply(func, axis=1)
nrel.head()

ValueError: malformed node or string: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 264.0, 2843.0, 5097.0, 6196.0, 7248.0, 7482.0, 7371.0, 6166.0, 4778.0, 1500.0, 153.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 125.0, 1967.0, 4732.0, 6336.0, 5284.0, 2705.0, 6934.0, 6304.0, 3985.0, 2625.0, 327.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 65.0, 1591.0, 4241.0, 2055.0, 6173.0, 3705.0, 3892.0, 6080.0, 4412.0, 455.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 254.0, 2928.0, 4954.0, 6556.0, 7528.0, 7615.0, 7747.0, 6843.0, 5373.0, 3081.0, 434.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 772.0, 4777.0, 6354.0, 6823.0, 7524.0, 6200.0, 4665.0, 4776.0, 1359.0, 144.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 170.0, 2366.0, 3422.0, 6269.0, 4106.0, 4797.0, 7191.0, 6516.0, 5124.0, 3053.0, 458.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 247.0, 2776.0, 4635.0, 3214

In [ ]:
# Rejoin aggregated data to other useful columns


SyntaxError: invalid syntax (2137817342.py, line 2)

In [ ]:
power_density = 160   # 160 W/m^2 (Rooftop Solar Photovoltaic Technical Potential in the United States: A Detailed Assessment)

# Convert hourly_capacity_factor (kWh) to kWh/m^2
grouped = grouped.apply(lambda x: )

In [ ]:
vc = pd.read_csv("hourly ten counties 2023-04-01 to 2025-04-09.csv")
vc.head()

,name,datetime,temp,feelslike,dew,humidity,precip,precipprob,preciptype,snow,...,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations
0,"Maricopa County, AZ",2023-04-01T00:00:00,51.5,51.5,29.4,42.42,0.0,0.0,NaN,0.0,...,1019.3,5.7,8.5,0.0,0.0,0,10,Clear,clear-night,"KLUF,72278023183,KPHX,KGXF,KBXK,72278523111"
1,"Maricopa County, AZ",2023-04-01T01:00:00,52.3,52.3,30.0,42.20,0.0,0.0,NaN,0.0,...,1018.8,5.7,9.9,0.0,0.0,0,10,Clear,clear-night,"KLUF,72278023183,KPHX,KGXF,KBXK,72278523111"
2,"Maricopa County, AZ",2023-04-01T02:00:00,50.5,50.5,30.0,45.05,0.0,0.0,NaN,0.0,...,1018.6,3.2,9.7,0.0,0.0,0,10,Clear,clear-night,"KLUF,72278023183,KPHX,KGXF,KBXK,72278523111"
3,"Maricopa County, AZ",2023-04-01T03:00:00,46.5,42.9,30.1,52.67,0.0,0.0,NaN,0.0,...,1018.2,3.2,9.8,0.0,0.0,0,10,Clear,clear-night,"KLUF,72278023183,KPHX,KGXF,KBXK,72278523111"
4,"Maricopa County, AZ",2023-04-01T04:00:00,45.9,42.5,30.1,53.86,0.0,0.0,NaN,0.0,...,1017.7,3.2,9.7,0.0,0.0,0,10,Clear,clear-night,"KLUF,72278023183,KPHX,KGXF,KBXK,72278523111"


solarenergy is measured in MJ/(m^2).

In [ ]:
# Convert solarenergy to from MJ/m^2 to kWh/m^2
# 1 kWh = 3.6 MJ
vc["solarenergy"] = vc["solarenergy"].apply(lambda x: x / 3.6)